# Open Exploration Rijkswaterstaat Reference Policy, Experiments Notebook
This notebook runs experiments for the reference policy for the problem formulation of Rijkswaterstaat. The results are analysed in the notebook 'Rijks_base_scenario.ipynb'

***

**Content**
1. Importing packages
2. Outcomes and experiments

***

### 1 | Importing packages
Start with importing the packages that are needed to complete the open exlporation. This includes more general packages and more specific packages from, for instance, the EMA workbench.

***

In [1]:
#Import necessary packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import altair as alt

In [2]:
#Import necessary packages, functions
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter, save_results, load_results)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.analysis import feature_scoring, prim, dimensional_stacking, pairs_plotting
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
import ema_workbench.em_framework.samplers
from SALib.analyze import sobol
from SALib.sample import saltelli, morris, fast_sampler
import SALib.util.results
from ema_workbench.analysis import regional_sa
from numpy.lib import recfunctions as rf


from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation

ema_logging.log_to_stderr(ema_logging.INFO)

def sum_over(*args):
    return sum(args)

### 2 | Outcomes and experiments

This part includes the access to outcomes, using problem formulation 3. A base scenario 0 is creates with no policy lever implemented. After this, the next step is to run 10000 experiments with a MultiprocessingEvaluator, with the base scenario without any policy measures. Results are saved.

***

In [3]:
#Get dike_model with problem formulation 3 
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [4]:
for outcome in dike_model.outcomes:
    print(outcome)

ScalarOutcome('A.1 Total Costs', variable_name=['A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'], function=<function sum_over at 0x0000029D9910F040>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=['A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'], function=<function sum_over at 0x0000029D9910F040>)
ScalarOutcome('A.2 Total Costs', variable_name=['A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'], function=<function sum_over at 0x0000029D9910F040>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=['A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'], function=<function sum_over at 0

In [7]:
# Create pace scenario without policy lever
base_policy = [Policy("policy", **{lever.name: 0 for lever in dike_model.levers})] 

In [8]:
# Run experiments
N_scen = 10000
policy = base_policy

In [14]:
#Run the model through the workbench.
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(N_scen, policy)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 10000 scenarios * 1 policies * 1 model(s) = 10000 experiments
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] 3000 cases completed
[MainProcess/INFO] 4000 cases completed
[MainProcess/INFO] 5000 cases completed
[MainProcess/INFO] 6000 cases completed
[MainProcess/INFO] 7000 cases completed
[MainProcess/INFO] 8000 cases completed
[MainProcess/INFO] 9000 cases completed
[MainProcess/INFO] 10000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [15]:
#The results of these experiments are saved in the below path, in the results folder. 
#These results can be analysed in the notebook 'RWS_OpenExploration_RefPolicy_Analysis.ipynb'
#In that notebook, the results of 100.000 experiments are analysed.
save_results(results, './results/base_scenario_rijks_.tar.gz')  

[MainProcess/INFO] results saved successfully to C:\Users\olivi\Documents\GitHub\Model_based\final assignment\results\base_scenario_rijks_.tar.gz
